### Notebook responsável por concentrar os métodos de análises

In [ ]:
import pandas as pd

In [1]:
def vencedor(diff):
    if diff > 0:
        return 'CASA'
    elif diff == 0:
        return 'EMPATE'
    else:
        return 'VISITANTE'

In [ ]:
def aplica_transformacoes_dataframe_partidas(df_match):
    df_match['diff_goals'] = df_match['home_goal'] - df_match['visitor_goal']
    df_match['vencedor'] = df_match['diff_goals'].map(vencedor)
    df_match['datetime'] = pd.to_datetime(df_match['date'])
    return df_match